<a href="https://colab.research.google.com/github/oloeye/captcha_crack/blob/master/tf2_captcha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install captcha
!pip install -U tensorflow-gpu==2.0.0a0

### 导入相应的包


In [0]:
# -*-  conding:utf-8 -*-
from captcha.image import ImageCaptcha
import matplotlib.pyplot as plt # plt 用于显示图片
import random
from PIL import Image
import numpy as np
import tensorflow as tf


print(tf.__version__)
print(tf.keras.__version__)
print(tf.test.is_gpu_available())

2.0.0-alpha0
2.2.4-tf
True


### 验证码的生成及处理

In [0]:
number = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z']
ALPHABET = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            'V', 'W', 'X', 'Y', 'Z']
SAVE_PATH = "/content/gdrive/My Drive/deep_learning/ml_models/"
CHAR_SET = number + alphabet + ALPHABET
CHAR_SET_LEN = len(CHAR_SET)
IMAGE_HEIGHT = 60
IMAGE_WIDTH = 160

# 随机生成文本
def random_captcha_text(char_set = None,captcha_size=4):
  if char_set is None:
    char_set = number + alphabet + ALPHABET
    
  captcha_text = []
  for _ in range(captcha_size):
    c = random.choice(char_set)
    captcha_text.append(c)
  return captcha_text

# 生成验证码图片
def gen_captcha_text_and_image(width=160, height=60, char_set=CHAR_SET):
  image = ImageCaptcha(width=width, height=height)

  captcha_text = random_captcha_text(char_set)
  captcha_text = ''.join(captcha_text)

  captcha = image.generate(captcha_text)

  captcha_image = Image.open(captcha)
#   im_show(captcha_image)
  captcha_image = np.array(captcha_image)
  
  return captcha_text, captcha_image


def im_show(img):
  plt.imshow(img) # 显示图片
  plt.axis('off') # 不显示坐标轴
  plt.show()

# 生成验证码

text, image = gen_captcha_text_and_image(char_set=CHAR_SET)

MAX_CAPTCHA = len(text)
print('CHAR_SET_LEN=', CHAR_SET_LEN, ' MAX_CAPTCHA=', MAX_CAPTCHA)
  

CHAR_SET_LEN= 62  MAX_CAPTCHA= 4


In [0]:
'''挂载到 google drive'''
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

def convert2gray(img):
    if len(img.shape) > 2:
        gray = np.mean(img, -1)
        return gray
    else:
        return img


def text2vec(text):
    vector = np.zeros([MAX_CAPTCHA, CHAR_SET_LEN])
    for i, c in enumerate(text):
        idx = CHAR_SET.index(c)
        vector[i][idx] = 1.0
    return vector


def vec2text(vec):
    text = []
    for i, c in enumerate(vec):
        text.append(CHAR_SET[c])
    return "".join(text)


def get_next_batch(batch_size=128):
    batch_x = np.zeros([batch_size, IMAGE_HEIGHT, IMAGE_WIDTH, 1])
    batch_y = np.zeros([batch_size, MAX_CAPTCHA, CHAR_SET_LEN])

    def wrap_gen_captcha_text_and_image():
        while True:
            text, image = gen_captcha_text_and_image(char_set=CHAR_SET)
            if image.shape == (60, 160, 3):
                return text, image

    for i in range(batch_size):
        text, image = wrap_gen_captcha_text_and_image()
#         im_show(image)
        image = tf.reshape(convert2gray(image), (IMAGE_HEIGHT, IMAGE_WIDTH, 1))
        batch_x[i, :] = image
        batch_y[i, :] = text2vec(text)

    return batch_x, batch_y


def crack_captcha_cnn():
    '''使用 vgg16 两个 卷积 一个池化'''
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Conv2D(32, (3, 3)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.Conv2D(128, (5, 5)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))   

    model.add(tf.keras.layers.Conv2D(256, (5, 5)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.Conv2D(256, (3, 3)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))
    
    model.add(tf.keras.layers.Conv2D(64, (5, 5)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.Conv2D(32, (3, 3)))
    model.add(tf.keras.layers.PReLU())
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.MaxPool2D((2, 2), strides=2))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(MAX_CAPTCHA * CHAR_SET_LEN))
    model.add(tf.keras.layers.Reshape([MAX_CAPTCHA, CHAR_SET_LEN]))
    model.add(tf.keras.layers.Softmax())

    return model


def train():
    try:
        model = tf.keras.models.load_model( SAVE_PATH+ 'model')
    except Exception as e:
        print('#######Exception', e)
        model = crack_captcha_cnn()

    model.compile(optimizer='Adam',
                  metrics=['accuracy'],
                  loss='categorical_crossentropy')

    for times in range(1000):
        batch_x, batch_y = get_next_batch(1024)
        print('times=', times, ' batch_x.shape=', batch_x.shape, ' batch_y.shape=', batch_y.shape)
        model.fit(batch_x, batch_y, epochs=5)
#         print("y预测=\n", np.argmax(model.predict(batch_x), axis=2))
#         print("y实际=\n", np.argmax(batch_y, axis=2))

        if 0 == times % 10:
            print("save model at times=", times)
            # 使用工具colab的接口挂载google drive目录，这样可以从外部获取数据并且可以把训练好的模型保存在google drive上
            model.save(SAVE_PATH + 'model')
     


def predict(batch_num = 1000):
    model = tf.keras.models.load_model(SAVE_PATH + 'model')
    success = 0
    for _ in range(batch_num):
        data_x, data_y = get_next_batch(1)
        prediction_value = model.predict(data_x)
        data_y = vec2text(np.argmax(data_y, axis=2)[0])
        prediction_value = vec2text(np.argmax(prediction_value, axis=2)[0])

        if data_y.upper() == prediction_value.upper():
            print("y预测=", prediction_value, "y实际=", data_y, "预测成功。")
            success += 1
        else:
            print("y预测=", prediction_value, "y实际=", data_y, "预测失败。")

    print("预测", batch_num, "次", "成功率 =", success / batch_num)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [6]:
# train()
predict(batch_num=100)

# !ls sample_data
# !rm -f sample_data/model
# !ls sample_data

W0617 07:31:04.711018 140333098760064 hdf5_format.py:266] Sequential models without an `input_shape` passed to the first layer cannot reload their optimizer state. As a result, your model isstarting with a freshly initialized optimizer.


y预测= xmy2 y实际= xmy2 预测成功。
y预测= JPmD y实际= JPmD 预测成功。
y预测= 0f7c y实际= Of7c 预测失败。
y预测= 69JU y实际= 69JU 预测成功。
y预测= D1a5 y实际= D1a5 预测成功。
y预测= AQS5 y实际= AQS5 预测成功。
y预测= fekV y实际= fekV 预测成功。
y预测= J9aW y实际= J9aW 预测成功。
y预测= MQGJ y实际= MQGJ 预测成功。
y预测= ThNu y实际= ThNu 预测成功。
y预测= Kyem y实际= Kyem 预测成功。
y预测= ySiR y实际= ySiR 预测成功。
y预测= FJ9P y实际= FJ9P 预测成功。
y预测= P2Q1 y实际= P2Q1 预测成功。
y预测= kFBD y实际= kFBD 预测成功。
y预测= oUJc y实际= oUJc 预测成功。
y预测= qyR2 y实际= qyR2 预测成功。
y预测= bIL2 y实际= bIL2 预测成功。
y预测= Ma7Z y实际= Ma7Z 预测成功。
y预测= jt6H y实际= jt6H 预测成功。
y预测= 6wVT y实际= 6wVT 预测成功。
y预测= P1io y实际= P1io 预测成功。
y预测= ShlG y实际= ShlG 预测成功。
y预测= tILI y实际= tILI 预测成功。
y预测= zIVM y实际= zIvM 预测成功。
y预测= vavu y实际= vavu 预测成功。
y预测= Plg6 y实际= Plg6 预测成功。
y预测= PVnO y实际= Pvn0 预测失败。
y预测= QmEo y实际= QmEo 预测成功。
y预测= PoEO y实际= PoEO 预测成功。
y预测= Pb3E y实际= Pb3E 预测成功。
y预测= 4V34 y实际= 4V34 预测成功。
y预测= kzNZ y实际= kzNZ 预测成功。
y预测= EzcP y实际= EzcP 预测成功。
y预测= Jaqs y实际= Jaqs 预测成功。
y预测= oV2M y实际= Ov2M 预测成功。
y预测= ODyH y实际= 0DyH 预测失败。
y预测= nTXZ y实际= nTXZ 预测成功。
y预测= WuM1 y实